In [ ]:
! python3 -m pip install --upgrade tensorrt
! pip3 install matplotlib

In [ ]:
import tensorflow as tf
import time
import matplotlib.pyplot as plt

# Configuration
matrix_size = 1000  # Increase this value for larger tensors (takes longer time)

def matrix_multiply_cpu():
    # Create two large random matrices
    matrix_a = tf.random.normal(shape=(matrix_size, matrix_size))
    matrix_b = tf.random.normal(shape=(matrix_size, matrix_size))

    # Perform matrix multiplication on CPU
    start_time = time.time()
    result = tf.matmul(matrix_a, matrix_b)
    end_time = time.time()
    return end_time - start_time

def matrix_multiply_gpu():
    # Create two large random matrices
    matrix_a = tf.random.normal(shape=(matrix_size, matrix_size))
    matrix_b = tf.random.normal(shape=(matrix_size, matrix_size))

    # Perform matrix multiplication on GPU
    with tf.device('/GPU:0'):
        start_time = time.time()
        result = tf.matmul(matrix_a, matrix_b)
        end_time = time.time()
    return end_time - start_time

# Main function
def main():
    cpu_times = []
    gpu_times = []

    for _ in range(5):  # Repeat the operation multiple times for better comparison
        cpu_time = matrix_multiply_cpu()
        gpu_time = matrix_multiply_gpu()

        cpu_times.append(cpu_time)
        gpu_times.append(gpu_time)

    # Print the average times
    print("Average time for CPU:", sum(cpu_times) / len(cpu_times), "seconds")
    print("Average time for GPU:", sum(gpu_times) / len(gpu_times), "seconds")

    # Plot the results
    plt.bar(["CPU", "GPU"], [sum(cpu_times) / len(cpu_times), sum(gpu_times) / len(gpu_times)])
    plt.xlabel("Device")
    plt.ylabel("Average Time (s)")
    plt.title("Performance Comparison: CPU vs. GPU")
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Check if TensorFlow is using the GPU
print("TensorFlow is using GPU:", tf.test.is_gpu_available())

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data to have zero mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 50
batch_size = 16
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# Make predictions on the test set
y_pred_probabilities = model.predict(X_test)
y_pred = tf.argmax(y_pred_probabilities, axis=1)

# Calculate and print the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy (calculated):", test_accuracy)